In [35]:
import auth
import requests
import base64
import json
import pandas as pd
pd.set_option('display.max_columns', None)  
flickr = auth.connect()


In [20]:
df = pd.read_csv('../us_cities.csv')


top_cities = ["New York", "Los Angeles", "Las Vegas", "Orlando", "Miami", "San Francisco"]

df_top = df[df['CITY'].isin(top_cities)]
df_top = df_top.reset_index()  

In [21]:
cities= {
   "New York":{
      "latitude":40.730610,
      "longitude":-73.935242
   },
   "Los Angeles":{
      "latitude":34.052235,
      "longitude":-118.243683
   },
   "Las Vegas":{
      "latitude":36.188110,
      "longitude":-115.176468
   },
   "Orlando":{
      "latitude":28.538336,
      "longitude":-81.379234
   },
   "Miami":{
      "latitude":25.761681,
      "longitude":-80.191788
   },
   "San Francisco":{
      "latitude":37.773972,
      "longitude":-122.431297
   }
}


Fields needed for each photo:
id: unique ID of the uploaded photo (pid).
owner-id: unique user ID of the person who uploaded the photo (up).
longitude: (geotag information) x coordinate (xp).
latitude: (geotag information) y coordinate (yp).
dates-taken: date and time when the photo was taken (tp).
dates-posted: date and time when the photo was uploaded.

In [30]:
for k,v in cities.items():
    data=[]
    pages = 8
    city = k
    long = v['longitude']
    lat = v['latitude']
    filename= city+'-photos.csv'

    for p in range(1, pages+1):
        
        photos = flickr.photos.search(lat=lat, lon=long, accuracy=11, radius = 20, radius_units='mi',min_date_taken='2010-01-01', max_date_taken='2023-12-31', extras="geo, tags, date, date_upload, date_taken, owner_name", page=p)

        df_photos = pd.DataFrame(photos['photos']['photo'])
        df_photos = df_photos.drop(['secret', 'farm', 'server', 'ispublic', 'isfriend', 'isfamily', 'datetakengranularity', 'datetakenunknown'], axis=1)
        df_photos = df_photos.drop_duplicates()
        data.append(df_photos)
    data = pd.concat(data)
    data.to_csv(filename)


 

In [31]:
df1= pd.read_csv('New York-photos.csv')

In [36]:
df1

,Unnamed: 0,id,owner,title,dateupload,datetaken,ownername,tags,latitude,longitude,accuracy,context,place_id,woeid,geo_is_public,geo_is_contact,geo_is_friend,geo_is_family
0,0,52745497042,197474324@N05,CZ0_8400,1678754450,2023-02-22 21:46:22,carlito_13,nyc newyork ny manhattan,40.765062,-73.978794,13,0,NaN,6944375,1,0,0,0
1,1,52745309374,95723449@N06,9/11 Memorial Park,1678724442,2022-10-03 15:06:07,Martinaway,newyorkcity nyc ny newyork manhattan financial...,40.711269,-74.013323,16,0,NaN,5108135,1,0,0,0
2,2,52743307282,7735863@N05,"wave hill, 3/12/23",1678675769,2023-03-12 15:55:48,hollow sidewalks,bronx newyorkcity nyc garden gardens wavehill ...,40.900900,-73.922576,16,0,NaN,7307345,1,0,0,0
3,3,52743235216,22078864@N03,Midtown Across the Lake (ND 1000 Filter),1678654067,2023-03-05 15:13:27,jamesconradshea,NaN,40.777513,-73.971978,16,0,NaN,5128594,1,0,0,0
4,4,52742345452,7702423@N04,Hawaiian Airlines A330-200s - Plane watching a...,1678645286,2022-11-05 10:23:16,cseeman,twahotelplanewatching johnfkennedyinternationa...,40.642236,-73.773880,14,0,NaN,5139287,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,245,51778063437,95506972@N02,IMG_1962,1640620354,2021-11-20 19:10:56,stickerkitty's map,NaN,40.753200,-73.992767,16,0,NaN,8410791,1,0,0,0
1996,246,51777634895,26946559@N02,IMG_8014.jpg,1640539886,2021-12-25 19:03:14,kevinrubin,newyorkcity streetphotography street nyc newyo...,40.759238,-73.977312,16,0,NaN,6930480,1,0,0,0
1997,247,51776398786,68776679@N05,#294,1640528761,2021-12-19 09:09:25,mario poli corsi,mariopolicorsi canon eos 700d fisheye samyang ...,40.760045,-73.979144,16,0,NaN,6930480,1,0,0,0
1998,248,51772991433,39326896@N05,HTP11188,1640371000,2021-12-24 09:01:23,Vinny Schiano,newark newjersey unitedstates,40.700425,-74.135048,16,0,NaN,5102094,1,0,0,0


In [33]:
photos_by_user = df1.groupby("owner")['id'].count().sort_values(ascending= False)

In [34]:
photos_by_user

owner
7702423@N04      87
62596395@N00     81
11161183@N06     70
46681937@N00     68
10485077@N06     42
                 ..
56406582@N00      1
56592697@N03      1
196841812@N06     1
56904012@N07      1
63659356@N07      1
Name: id, Length: 450, dtype: int64

In [37]:
df_user = df1[df1['owner']=='7702423@N04']

In [39]:
df_user.sort_values(by='datetaken', inplace = True) 


/Users/ratikabhuwalka/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [43]:
df_user.head(80)

,Unnamed: 0,id,owner,title,dateupload,datetaken,ownername,tags,latitude,longitude,accuracy,context,place_id,woeid,geo_is_public,geo_is_contact,geo_is_friend,geo_is_family
256,6,52626677409,7702423@N04,Challenge Airlines 747s - Plane watching at th...,1673707343,2022-11-02 14:39:27,cseeman,twahotelplanewatching johnfkennedyinternationa...,40.642236,-73.773880,14,0,NaN,5139287,1,0,0,0
339,89,52590499701,7702423@N04,Eva Air Cargo 777-F5E (B-16783) - Plane watchi...,1672194910,2022-11-02 14:44:00,cseeman,twahotelplanewatching johnfkennedyinternationa...,40.643441,-73.770232,14,0,NaN,5139287,1,0,0,0
254,4,52626424607,7702423@N04,Emirates A380-800 (A6-EUR) - Plane watching at...,1673724614,2022-11-02 14:45:15,cseeman,twahotelplanewatching johnfkennedyinternationa...,40.642236,-73.773880,14,0,NaN,5139287,1,0,0,0
567,67,52507200419,7702423@N04,Emirates A380-800 (A6-EUR) - Plane watching at...,1668735335,2022-11-02 15:00:14,cseeman,twahotelplanewatching johnfkennedyinternationa...,40.644060,-73.778901,14,0,NaN,5139287,1,0,0,0
554,54,52509093422,7702423@N04,"TWA Hotel (JFK Airport, New York, NY - Novembe...",1668856090,2022-11-02 15:19:43,cseeman,twahotel twaflightcenter transworldairlinesfli...,40.645655,-73.777871,14,0,NaN,5139287,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,61,52602184764,7702423@N04,West Bank Light from the Anthem of the Seas - ...,1672657865,2022-11-06 16:55:52,cseeman,anthemoftheseas royalcaribbean royalcaribbeans...,40.537925,-74.043012,16,0,NaN,5137499,1,0,0,0
312,62,52602437233,7702423@N04,West Bank Light from the Anthem of the Seas - ...,1672657880,2022-11-06 16:59:19,cseeman,anthemoftheseas royalcaribbean royalcaribbeans...,40.537925,-74.043012,16,0,NaN,5137499,1,0,0,0
435,185,52552198650,7702423@N04,Romer Shoal Light from the Anthem of the Seas ...,1670551322,2022-11-06 17:01:23,cseeman,anthemoftheseas royalcaribbean royalcaribbeans...,40.514425,-74.011802,11,0,NaN,5098133,1,0,0,0
525,25,52524477311,7702423@N04,Pilot Boats During Embarkation Day on the Anth...,1669475214,2022-11-06 17:40:36,cseeman,anthemoftheseas royalcaribbean royalcaribbeans...,40.544434,-73.997726,11,0,NaN,5137499,1,0,0,0
